# Notebook Principal: Pipeline de Ensembles

Este notebook organiza o fluxo para ensembles por modelo (tile, imagem, paciente) e entre modelos.

Observação: nome e estrutura podem ser ajustados conforme sua confirmação.

## 1. Introdução e Objetivos
- Contexto do projeto
- Objetivos dos ensembles por modelo e entre modelos
- Organização dos dados e saídas

## 2. Configurações e Parâmetros
Defina os parâmetros globais do pipeline (modelos, tipo de ensemble, métrica de peso, diretórios).

In [ ]:
import os
from modules import BetweenModelsEnsembler, PerModelEnsembler

# Configurações padrão (ajuste conforme necessário)
MODELOS_ENTRE = ['GGNet', 'MOBNet', 'SHFFNet']
ENSEMBLE_TYPE = 'hard_voting'  # 'hard_voting' | 'soft_voting' | 'weighted'
WEIGHT_METRIC = 'f1_macro'     # usado apenas com 'weighted'

# Diretórios base
ROOT = os.path.dirname(os.path.abspath(''))
TABLES_DIR = os.path.join(ROOT, 'tables')
ENTRE_MODELOS_OUT = os.path.join(TABLES_DIR, 'Ensemble_Between_Models')
os.makedirs(ENTRE_MODELOS_OUT, exist_ok=True)


## 3. Carregamento de Dados
Explique/implemente como localizar os CSVs de entrada. Para o PerModelEnsembler,
forneça a lista de caminhos para os CSVs de cada fold do modelo escolhido.

In [ ]:
# Exemplo: localizar CSVs de um modelo específico
import glob

MODEL_NAME = 'GGNet'
MODEL_CSVS = sorted(glob.glob(os.path.join(TABLES_DIR, MODEL_NAME, '**', '*.csv'), recursive=True))
print(f'Encontrados {len(MODEL_CSVS)} CSVs para o modelo {MODEL_NAME}.')
MODEL_CSVS[:5]

## 4. Ensemble por Modelo (Tile → Image → Patient)
Execute o ensemble por modelo nas três etapas.

In [ ]:
per_model = PerModelEnsembler(model_name=MODEL_NAME, ensemble_type=ENSEMBLE_TYPE, weight_metric=WEIGHT_METRIC)
tile_csv, tile_metrics = per_model.run_tile_level(MODEL_CSVS)
print('Tile level salvo em:', tile_csv)
image_csv, image_metrics = per_model.run_image_level(MODEL_CSVS)
print('Image level salvo em:', image_csv)
patient_csv, patient_metrics = per_model.run_patient_level(MODEL_CSVS)
print('Patient level salvo em:', patient_csv)

## 5. Ensemble Entre Modelos (Tile, Image, Patient)
Orquestre o ensemble entre diferentes modelos.

In [ ]:
between = BetweenModelsEnsembler(
    base_models_parent_directory=TABLES_DIR,
    ensemble_save_output_base=ENTRE_MODELOS_OUT,
    models_to_include=MODELOS_ENTRE,
    ensemble_type=ENSEMBLE_TYPE,
    weight_metric=WEIGHT_METRIC,
)
img_csv, img_metrics = between.run_image_level()
tile_csv, tile_metrics = between.run_tile_level()
pat_csv, pat_metrics = between.run_patient_level()
print(img_csv, tile_csv, pat_csv)

## 6. Avaliação de Métricas e Visualizações
Carregue os JSONs de métricas e gere gráficos (matriz de confusão, ROC, etc.).

In [ ]:
import json

def load_metrics(path):
    with open(path, 'r') as f:
        return json.load(f)

print('Metrics (per-model tile):', load_metrics(tile_metrics))
print('Metrics (entre-modelos image):', load_metrics(img_metrics))

## 7. Exportação de Resultados
Descreva como exportar os CSVs e plots para repositórios de resultados.

## 8. Próximos Passos
- Ajustar nome final do notebook
- Refino dos utilitários e visualizações
- Automatização de descoberta de CSVs por modelo